# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

# Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='~/../images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='~/../images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

# Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!


# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import helper_functions as hf
import matplotlib.pyplot as plt
import itertools
from matplotlib.pylab import rcParams
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace import sarimax
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
data_wide = pd.read_csv('./zillow_data.csv')

In [ ]:
data_wide.head()

In [ ]:
data_long = hf.melt_data_v2(data_wide)
data_long['time'] = pd.to_datetime(data_long['time'], format='%Y-%m-%d')
data_long['RegionName'] = data_long['RegionName'].astype('str')

data_long.set_index(keys='time', inplace=True)
data_long.head()

In this project, we will be looing at zip codes in Florida, specifically areas near Orlando

In [ ]:
#separate out areas within the state of Florida
df_fl = data_long.loc[data_long.State=='FL']
df_fl.info()

In [ ]:
df_fl.isna().sum()

In [ ]:
df_fl.info()
df_fl.head()

# Step 2: Data Preprocessing

# Step 3: EDA and Visualization

In [ ]:
from matplotlib import rc

font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 12}

rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

In [ ]:
# In the state array, there are 51 entries.
data_long.State.unique().shape

In [ ]:
df_fl.head()

In [ ]:
#Looking at each state to see the total means over the years
plt.figure(figsize=(21,13))
for state in data_long.State.unique():
    state_data = data_long.loc[(data_long.State==state), ['value']].resample('MS').sum()
    plt.plot(state_data, label=state)
    
plt.title('Sum of monthly means of US states housing market')
plt.legend()
plt.show()

In [ ]:
# loop to print out the counties for the state of Florida
plt.figure(figsize=(21,13))
for county in df_fl.CountyName.unique():
    county_data = df_fl.loc[(df_fl.CountyName==county), ['value']].resample('MS').mean()
    plt.plot(county_data, label=county)
plt.legend()
plt.show()

In [ ]:
# Florida monthly means ovre the years
florida_monthly = df_fl.groupby(pd.Grouper(freq='MS'))
florida_monthly.value.mean().plot(figsize=(15,5))

In [ ]:
# Separate out the areas that are part of Orange County, Florida BY YEAR 2011
orange_county = df_fl.loc[(df_fl.CountyName=='Orange'), ['value']].resample('MS').mean()
plt.figure(figsize=(15,6))
plt.plot(orange_county)
plt.legend()
plt.show()

# df_fl_2011.loc[(df_fl_2011.CountyName=='Orange'), ['value']].resample('MS').mean()

Overall, Orange county's prices greatly resemble that of the entire state of Florida.

Below, we will take a look at the zip codes that make up Orange county.

In [ ]:
zipcodes = df_fl.loc[(df_fl.CountyName=='Orange')].RegionName.unique()
plt.figure(figsize=(15,5))
for zip_code in zipcodes:
    area = df_fl.loc[(df_fl.RegionName==zip_code), ['value']].resample('MS').mean()
    plt.plot(area, label=zip_code)
    plt.legend(loc='best')
plt.show()

Based on our graph below, there appears to be some seasonality that is occurring within the housing market for orange county as time passes by.

In [ ]:
## Looking at difference by year
# Look at the distribution of the diffs and look at the one with the smallest standard deviation

plt.gcf().autofmt_xdate()

orange_county_diff = orange_county.diff(periods=1)
rcParams['figure.figsize'] = (12, 5)
plt.plot(orange_county_diff)

rcParams['figure.figsize'] = (12, 5)
plot_acf(orange_county['value'], title='Orange County Auto Correlation');
plot_pacf(orange_county['value'], title='Orange County Partial Auto Correlation');

Based on what we see in our partial correlation plot, there is a high negative correlation somewhere between 220 - 245 lags.

This high negative appears at lag = 180 months

# Step 4: Reshape from Wide to Long Format

In [ ]:
orange_county.head()

# Step 5: ARIMA Modeling

Before getting into the ARIMA modeling, combinations for the model needs to be created.  
Here, the parameters for all combinations of seasons are also added to our values for seasonal & non seasonal arima modeling.

In [ ]:
p = d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
pdqs = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

## Generate combinations of SARIMA modeling with different seasonalities
# pdqs = []    
# for i in range(0,13):
#     for x in pdq:
#         pdqs.append((x[0], x[1], x[2], i))

pdqs

In [ ]:
## NEED TO TURN THIS INTO A FUNCTION THAT I CAN PASS
## EACH ZIP CODE INTO AS A DATAFRAME, 
ans = []
for comb in pdq:
    for combs in pdqs:
        try:
            mod = hf.model_SARIMA(df=orange_county, order=comb, s_order=combs)
            ans.append([comb, combs, mod.aic, mod.bic])
            print('ARIMA {} x {} : AIC Calculated ={}, BIC Calculated ={}'.format(comb, combs, mod.aic, mod.bic))
        except:
            continue

After running all of the possible combinations through the seasonal ARIMA model, the results of each combination was stored in a dataframe, so that we can easily search for the optimum model.

In [ ]:
ans_df = pd.DataFrame(ans, columns=['pdq', 'pdqs', 'aic', 'bic'])
ans_df

Here we see that the our best model has both the highest `aic`and `bic` scores.

In [ ]:
ans_df.loc[ans_df['aic'].idxmin()]

In [ ]:
ans_df.loc[ans_df['bic'].idxmin()]

Now that we have the results from our SARIMA model, we will now take the results with best AIC and BIC and pass it into our Model to see how ti performed compared to the others

Fitting ARIMA Time Series Model 

In [ ]:
### Get the results of our best parameters for our ARIMA model ###
order = ans_df.loc[ans_df['aic'].idxmin()]['pdq']
s_order = ans_df.loc[ans_df['aic'].idxmin()]['pdqs']
ARIMA_MODEL = hf.model_SARIMA(orange_county, order=order, s_order=s_order, print_table=True)

Now, we are going to take a look where some of the residuals are deviating from the standard deviation and attempt to create a batter model with

### Plottting residuals 

In [ ]:
hf.diagnostics_plot(model=ARIMA_MODEL)

Based on the results of the plot diagnostics, we see that our data is not normally distributed. From here, we will continue to further look into our model and attempt to improve the results by removing outliers and the residuals that are causing issues within our model.

## Making predictions using our model parameters 

In [ ]:
orange_county.idxmin()

In [ ]:
hf.one_step_ahead_forecast(df=orange_county, start_date='2014', end_date='2021', arima_model=ARIMA_MODEL)

### Dynamic forecasting of the data


In [ ]:
hf.dynamic_prediction(df=orange_county, start_date='2014', end_date='2021', arima_model=ARIMA_MODEL)

Based on the results of our dynamic prediction, the housign prices are forecast to steadily increase into 2021

## Findng the best zip code within Orange County

To find the best zip code within the given area, we wil use the following formula to calculate the return of investment:

$$\large R.O.I = \frac{(GFI - CoI)}{CoI}$$

- ROI = Return of Investment
- GFI = Gain from Investment
- CoI = Cost of Investment

Our Cost of Ivestment will be the average of 2017, since we do not have a complete dataset for 2018


To calculate GFI, we will take our cost of investment and subtract it from the average predicted means from 2018 to 2021.

We will then use the formula above to calculate the return of investment for each zip code observed

In [ ]:
## Zipcodes of Orange County, Florida
zipcodes

In [ ]:
ROI_list = []
model_list = []

## Loop to get each zip code and calculate return on ivestment ##
for code in zipcodes:
    zip_df = df_fl.loc[(df_fl.RegionName==code), ['value']].resample('MS').mean()
    zip_model = model_SARIMA(zip_df, order=order, s_order=s_order)

    pred = zip_model.get_prediction(start=pd.to_datetime('2014'), end=pd.to_datetime('2021'))

    ## Define the initial cost of investment as of 2017 ##
    cost_of_investment = zip_df['2017'].value.mean()

    ## Calculate gain from investmnt from 2018 up to 2021 ##
    gain_from_investment = pred.predicted_mean['2018':].mean()

    ## calculate Return of Investment for the observed zip code
    ROI = (gain_from_investment - cost_of_investment)/cost_of_investment
    ROI_list.append(ROI)
    model_list.append(zip_model)

In [ ]:
df_results = pd.DataFrame(data=list(zip(zipcodes, ROI_list, model_list)), columns=['zip_code','ROI','model'])

# Step 6: Interpreting Results

Based on the results of our model, the top 5 zip codes to purchase a house from 2018 -2021 are as follows:
    

In [ ]:
df_results.sort_values(by='ROI', ascending=False).drop('model', axis=1).head()

In [ ]:
df_results.columns

## Conclusion

Based on the results of our modeling and testing, the best zip code in Orange County to purchase a home is 32839.
The return on investment for this home is listed at 4.